## Import library, create train and test data

In [7]:
%reload_ext autoreload
%autoreload 2
import re
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.preprocessing import OneHotEncoder
from module.layers.Dense import Dense
from module.optimizer.Adam import Adam
from module.layers.RNN import RNN
from tensorflow import keras
from module.Sequential import Sequential


df = pd.read_csv("./dataset/DanhgiaSmartphone.csv")
data = df[["comment", "label"]].values
np.random.shuffle(data)
print(len(data)) # 113
train_data = data[:63]
test_data = data[:63]

113


# Build word embedding


In [8]:
# Hàm lấy word embedding cho 1 từ
# get embedding vector for a word
def get_wv(w):
    try:
        return w2v.wv[w]
    except KeyError:
        return w2v.wv["UNK"]

# Hàm tạo word embedding cho 1 tập dữ liệu 
# Get get embedding vector for the dataset
def word_embedding(sentences):
    sen_split = [[word for word in sen.lower().split()]for sen in sentences]
    x = []
    for s in sen_split:
        v = []
        for w in s:
            v.append(get_wv(w))
        x.append(v)
    x = keras.preprocessing.sequence.pad_sequences(x, padding="post", dtype="float32")
    return x

""" # Train word embedding model use Word2Vec
df = pd.read_csv("./dataset/Train.csv")
sen_embedding = df["comment"]

# Xóa dấu câu và số trong dataset dùng để xây dựng word embedding
# Remove special characters and digit
sen_embedding_clean = []
for sen in sen_embedding:
    clean_sen = re.sub(r'[^\w\s]', '', sen)
    clean_sen = re.sub(r'\d', '', clean_sen)
    sen_embedding_clean.append(clean_sen)

sen_embedding_clean = [[word for word in sen.lower().split()] for sen in sen_embedding_clean]
w2v = Word2Vec(sen_embedding_clean, vector_size=10)
unk_vector = np.random.randn(10)
w2v.wv.add_vector("UNK", unk_vector)
w2v.save("test.model")  """
w2v = Word2Vec.load("./warehouse/test.model")

x_train = word_embedding(train_data[:, 0])
y_train = OneHotEncoder(sparse_output=False).fit_transform(train_data[:, 1].reshape(-1, 1))
print(x_train.shape)
print(y_train.shape)

(63, 6, 10)
(63, 3)


## Build model

In [15]:
from module.layers.LSTM import LSTM

md = Sequential()
md.add(LSTM(32, active="tanh"))
md.add(Dense(3, active="softmax"))
md.compile(optimizer=Adam(lr=0.003, beta1=0.9, beta2=0.99999))
md.fit(X=x_train, y=y_train, batch_size=6, epochs=100)

Epoch 0  [==========]  loss: 0.3643, accuracy 42.86%
Epoch 1  [==========]  loss: 0.3563, accuracy 47.62%
Epoch 2  [==========]  loss: 0.3312, accuracy 49.21%
Epoch 3  [==========]  loss: 0.2967, accuracy 53.97%
Epoch 4  [==========]  loss: 0.2789, accuracy 53.97%
Epoch 5  [==========]  loss: 0.2726, accuracy 58.73%
Epoch 6  [==========]  loss: 0.2700, accuracy 57.14%
Epoch 7  [==========]  loss: 0.2700, accuracy 57.14%
Epoch 8  [==========]  loss: 0.2688, accuracy 57.14%
Epoch 9  [==========]  loss: 0.2674, accuracy 57.14%
Epoch 10 [==========]  loss: 0.2654, accuracy 55.56%
Epoch 11 [==========]  loss: 0.2640, accuracy 55.56%
Epoch 12 [==========]  loss: 0.2620, accuracy 57.14%
Epoch 13 [==========]  loss: 0.2602, accuracy 57.14%
Epoch 14 [==========]  loss: 0.2578, accuracy 60.32%
Epoch 15 [==========]  loss: 0.2554, accuracy 61.90%
Epoch 16 [==========]  loss: 0.2524, accuracy 63.49%
Epoch 17 [==========]  loss: 0.2490, accuracy 61.90%
Epoch 18 [==========]  loss: 0.2449, accuracy 

## Test model's accuracy

In [16]:
x_test = word_embedding(test_data[:, 0])
y_test = OneHotEncoder(sparse_output=False).fit_transform(test_data[:, 1].reshape(-1, 1))
pre, score = md.evalute(x_test, y_test)
print(f"Test accuracy: {score:.2f}")

Test accuracy: 0.95
